In [12]:
import geopandas as gpd
from shapely.geometry import Point

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')

# Define Harvard Square location
harvard_square = Point(-71.1189, 42.3736)
harvard_gdf = gpd.GeoDataFrame(geometry=[harvard_square], crs=parcels.crs)
harvard_gdf_proj = harvard_gdf.to_crs(epsg=26986)  # Massachusetts state plane

# Filter for commercial parcels
commercial_use_codes = ['300', '301', '302', '303', '304', '305', '310',
                        '320', '325', '326', '330', '340', '345', '346']
commercial_parcels = parcels[parcels['use_code'].astype(str).str.startswith(('3', '4'))]
commercial_parcels_proj = commercial_parcels.to_crs(epsg=26986)

# Create 500m buffer around Harvard Square in projected coordinates
harvard_buffer_proj = harvard_gdf_proj.buffer(500)  # 500 meters
result_parcels_proj = commercial_parcels_proj[commercial_parcels_proj.geometry.intersects(harvard_buffer_proj.iloc[0])]

# Convert back to original CRS
result_parcels = result_parcels_proj.to_crs(parcels.crs)

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} commercial parcels within 500m of Harvard Square")
print(f"Parcel IDs: {result_ids}")

Found 93 commercial parcels within 500m of Harvard Square
Parcel IDs: ['166-77', '168-36', '160-59', '160-14', '166-37', '165-55', '165-60', '165-34', '137-13', '162-29', '168-59', '160-64', '165-53', '133-36', '168-21', '166-52', '169-82', '169-100', '162-73', '133-56', '166-35', '169-84', '160-83', '169-86', '168-48', '133-15', '160-76', '160-72', '133-23', '168-49', '162-26', '168-33', '166-24', '168-20', '160-67', '162-66', '161-92', '160-69', '133-54', '160-58', '136-15', '160-66', '133-28', '162-36', '169-98', '162-17', '133-27', '133-52', '162-19', '160-77', '169-7', '168-13', '134-7', '133-49', '162-54', '159-2', '169-67', '169-46', '133-12', '160-71', '169-68', '168-64', '168-41', '169-42', '160-11', '133-51', '166-31', '162-68', '169-47', '169-102', '162-65', '169-99', '165-57', '162-67', '160-57', '168-60', '169-81', '160-48', '135-123', '160-85', '168-22', '169-50', '168-25', '162-18', '133-20', '162-64', '160-84', '170-39', '133-48', '169-93', '160-63', '166-33', '133-14']

In [11]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

# 1. LOAD DATA
parcels = gpd.read_file('cambridge_parcels.geojson')

# 2. DATA PREPARATION
# Define Harvard Square location
harvard_square = Point(-71.1189, 42.3736)

# Convert to projected CRS for accurate measurements
harvard_gdf = gpd.GeoDataFrame(geometry=[harvard_square], crs=parcels.crs)
harvard_gdf_proj = harvard_gdf.to_crs(epsg=26986)  # Massachusetts state plane

# Filter for commercial parcels
commercial_parcels = parcels[parcels['general_us'] == 'Commercial']
if len(commercial_parcels) == 0:
    commercial_use_codes = ['300', '301', '302', '303', '304', '305', '310',
                           '320', '325', '326', '330', '340', '345', '346']
    commercial_parcels = parcels[parcels['use_code'].astype(str).str.startswith(('3', '4'))]

commercial_parcels_proj = commercial_parcels.to_crs(epsg=26986)

# 3. CONSTRAINT IMPLEMENTATION
# Create 500m buffer around Harvard Square in projected coordinates
harvard_buffer_proj = harvard_gdf_proj.buffer(500)  # 500 meters

# Find commercial parcels within buffer
result_parcels_proj = commercial_parcels_proj[commercial_parcels_proj.geometry.intersects(harvard_buffer_proj.iloc[0])]

# Convert back to original CRS
result_parcels = result_parcels_proj.to_crs(parcels.crs)

# 4. RESULT FILTERING AND SORTING
# Sort by distance to Harvard Square (optional)
# Convert back to original CRS for display

# 5. DISPLAY RESULTS
print(f"Found {len(result_parcels)} commercial parcels within 500m of Harvard Square")
print(result_parcels[['ml', 'location', 'use_code']].head(10))

result_ids = result_parcels['ml'].tolist()
print(f"\nParcel IDs: {result_ids}")
print(f"Total parcels found: {len(result_ids)}")


Found 85 commercial parcels within 500m of Harvard Square
          ml                     location use_code
272   166-77                  5 Revere St      340
329   168-36                 1 Brattle Sq      346
337   160-59                21 Dunster St      345
380   160-14  1336-1362 Massachusetts Ave      345
405   166-37                  8 Revere St      340
492   165-55             14 University Rd      345
785   165-34             110 Mt Auburn St      300
945   162-29                  10 Eliot Sq      326
1213  168-59                48 Brattle St      345
1246  160-64             28-36 Brattle St      345

Parcel IDs: ['166-77', '168-36', '160-59', '160-14', '166-37', '165-55', '165-34', '162-29', '168-59', '160-64', '165-53', '133-36', '168-21', '166-52', '169-82', '169-100', '162-73', '166-35', '169-84', '160-83', '169-86', '168-48', '133-15', '160-76', '160-72', '133-23', '168-49', '162-26', '168-33', '166-24', '168-20', '160-67', '160-69', '160-58', '136-15', '160-66', '133-2

In [15]:
import geopandas as gpd
from shapely.geometry import Point

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')

# Define Harvard Square location (fixed coordinates)
harvard_square = Point(-71.1189, 42.3736)

# Create GeoDataFrame for Harvard Square
harvard_gdf = gpd.GeoDataFrame(geometry=[harvard_square], crs=parcels.crs)

# Convert to projected CRS for accurate distance measurement
harvard_gdf_proj = harvard_gdf.to_crs(epsg=26986)  # Massachusetts state plane
parcels_proj = parcels.to_crs(epsg=26986)  # Massachusetts state plane

# Define commercial use codes consistently
commercial_use_codes = ['300', '301', '302', '303', '304', '305', '310',
                       '320', '325', '326', '330', '340', '345', '346']

# Filter for commercial parcels using use code (more reliable than general_us)
commercial_parcels_proj = parcels_proj[parcels_proj['use_code'].astype(str).str.startswith(('3', '4'))]

# Create 500m buffer around Harvard Square in projected coordinates
harvard_buffer_proj = harvard_gdf_proj.buffer(500)  # Exactly 500 meters

# Find commercial parcels within 500m of Harvard Square
result_parcels_proj = commercial_parcels_proj[commercial_parcels_proj.geometry.intersects(harvard_buffer_proj.iloc[0])]

# Convert back to original CRS
result_parcels = result_parcels_proj.to_crs(parcels.crs)

# Sort by ml (parcel ID) for consistency in results
result_parcels = result_parcels.sort_values('ml')

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} commercial parcels within 500m of Harvard Square")
print(f"Parcel IDs: {result_ids}")

Found 93 commercial parcels within 500m of Harvard Square
Parcel IDs: ['133-12', '133-14', '133-15', '133-20', '133-23', '133-27', '133-28', '133-36', '133-48', '133-49', '133-51', '133-52', '133-54', '133-56', '134-7', '135-123', '136-15', '137-13', '159-2', '160-11', '160-14', '160-48', '160-57', '160-58', '160-59', '160-63', '160-64', '160-66', '160-67', '160-69', '160-71', '160-72', '160-76', '160-77', '160-83', '160-84', '160-85', '161-92', '162-17', '162-18', '162-19', '162-26', '162-29', '162-36', '162-54', '162-64', '162-65', '162-66', '162-67', '162-68', '162-73', '165-34', '165-53', '165-55', '165-57', '165-60', '166-24', '166-31', '166-33', '166-35', '166-37', '166-52', '166-77', '168-13', '168-20', '168-21', '168-22', '168-25', '168-33', '168-36', '168-41', '168-48', '168-49', '168-59', '168-60', '168-64', '169-100', '169-102', '169-42', '169-46', '169-47', '169-50', '169-67', '169-68', '169-7', '169-81', '169-82', '169-84', '169-86', '169-93', '169-98', '169-99', '170-39']

In [19]:
import geopandas as gpd

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')

# Define retail use codes consistently
retail_use_codes = ['320', '321', '322', '323', '324', '325', '326', '330', '331']

# Filter for retail parcels
retail_parcels = parcels[parcels['use_code'].astype(str).isin(retail_use_codes)]

# Filter for large retail parcels (>6000 sq ft)
large_retail_parcels = retail_parcels[retail_parcels['land_area'] > 6000]

# Sort by parcel ID for consistency
large_retail_parcels = large_retail_parcels.sort_values('ml')

# Display results
result_ids = large_retail_parcels['ml'].tolist()
print(f"Found {len(large_retail_parcels)} retail parcels larger than 4000 sq ft")
print(f"Parcel IDs: {result_ids}")

Found 80 retail parcels larger than 4000 sq ft
Parcel IDs: ['100-62', '105-68', '105-82', '106-124', '107-117', '107-9', '109-50', '110-91', '116-12', '120-48', '125-73', '128-63', '128-72', '134-33', '136-15', '140-148', '152-25', '156-25', '157-26', '16-11', '162-26', '162-54', '169-46', '169-47', '174-37', '175-37', '175-75', '175-84', '176-15', '179-87', '18-64', '18-65', '180-57', '182-88', '183-100', '184-159', '189-72', '191-65', '191-85', '192-121', '192-177', '196-154', '199-30', '199-31', '199-67', '1A-189', '1A-192', '1A-211', '20-75', '200-22', '21-121', '226-46', '229-117', '230-71', '234-178', '236-95', '252-172', '260-46', '260-76', '265B-26', '265B-28', '265B-61', '265C-25', '267E-234', '268B-15', '273-2', '273-20', '34-8', '70-91', '70-92', '74-1', '8-88', '81-100', '83-80', '84-101', '84-91', '90-155', '90-162', '92-88', '93-78']


In [20]:
import geopandas as gpd

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')
poi = gpd.read_file('cambridge_poi_processed.geojson')

# Convert to projected CRS for accurate distance measurement
parcels_proj = parcels.to_crs(epsg=26986)  # Massachusetts state plane

# Filter restaurants from POI data
restaurants = poi[poi['business_type'] == 'restaurant']
restaurants_proj = restaurants.to_crs(epsg=26986)  # Massachusetts state plane

# Count nearby restaurants for each parcel
def count_nearby_restaurants(parcel_geom, restaurant_geoms, buffer_distance=300):
    buffered_geom = parcel_geom.buffer(buffer_distance)
    count = sum(1 for rest_geom in restaurant_geoms if buffered_geom.contains(rest_geom))
    return count

# Calculate restaurant count for each parcel
restaurant_geoms = restaurants_proj['geometry'].tolist()
parcels_proj['nearby_restaurants'] = parcels_proj['geometry'].apply(
    lambda geom: count_nearby_restaurants(geom, restaurant_geoms)
)

# Filter for parcels with 0, 1, or 2 restaurants within 300m
result_parcels = parcels_proj[parcels_proj['nearby_restaurants'] <= 2]

# Convert back to original CRS
result_parcels = result_parcels.to_crs(parcels.crs)

# Sort by parcel ID for consistency
result_parcels = result_parcels.sort_values('ml')

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} parcels with ≤2 restaurants within 300m")
print(f"Parcel IDs: {result_ids[:30]}")  # Show first 30 due to potentially large result set

Found 2517 parcels with ≤2 restaurants within 300m
Parcel IDs: ['103-95', '108-49', '108-50', '108-60', '108-61', '108-62', '112-102', '112-104', '112-106', '112-110', '112-111', '112-112', '112-113', '112-114', '112-124', '112-125', '112-139', '112-140', '112-141', '112-142', '112-143', '112-23', '112-46', '112-66', '113-1', '113-13', '113-14', '113-15', '113-17', '113-18']


In [24]:
import geopandas as gpd

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')
poi = gpd.read_file('cambridge_poi_processed.geojson')

# Convert to projected CRS for accurate distance measurement
parcels_proj = parcels.to_crs(epsg=26986)  # Massachusetts state plane

# Filter restaurants from POI data
restaurants = poi[poi['business_type'] == 'restaurant']
restaurants_proj = restaurants.to_crs(epsg=26986)  # Massachusetts state plane

# Count nearby restaurants for each parcel
def count_nearby_restaurants(parcel_geom, restaurant_geoms, buffer_distance=800):
    buffered_geom = parcel_geom.buffer(buffer_distance)
    count = sum(1 for rest_geom in restaurant_geoms if buffered_geom.contains(rest_geom))
    return count

# Calculate restaurant count for each parcel
restaurant_geoms = restaurants_proj['geometry'].tolist()
parcels_proj['nearby_restaurants'] = parcels_proj['geometry'].apply(
    lambda geom: count_nearby_restaurants(geom, restaurant_geoms)
)

# Filter for parcels with 0, 1, or 2 restaurants within 800m
result_parcels = parcels_proj[parcels_proj['nearby_restaurants'] <= 2]

# Convert back to original CRS
result_parcels = result_parcels.to_crs(parcels.crs)

# Sort by parcel ID for consistency
result_parcels = result_parcels.sort_values('ml')

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} parcels with ≤2 restaurants within 800m")
print(f"Parcel IDs: {result_ids}")

Found 57 parcels with ≤2 restaurants within 500m
Parcel IDs: ['242-33', '242-34', '242-36', '242-38', '242A-100', '242A-102', '242A-105', '242A-107', '242A-109', '242A-110', '242A-111', '242A-112', '242A-113', '242A-115', '242A-116', '242A-126', '242A-127', '242A-131', '242A-148', '242A-158', '242A-159', '242A-160', '242A-161', '242A-162', '242A-59', '242A-68', '242A-72', '242A-73', '242A-75', '242A-91', '242A-99', '242B-999', '243-1', '266-1', '266-10', '266-12', '266-13', '266-14', '266-15', '266-16', '266-17', '266-18', '266-19', '266-2', '266-20', '266-21', '266-3', '266-35', '266-36', '266-37', '266-38', '266-4', '266-5', '266-6', '266-7', '266-8', '266-9']


In [26]:
import geopandas as gpd

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')
census = gpd.read_file('cambridge_census_cambridge_pct.geojson')

# Define commercial use codes consistently
commercial_use_codes = ['300', '301', '302', '303', '304', '305', '310',
                        '320', '325', '326', '330', '340', '345', '346']

# Filter for commercial parcels
commercial_parcels = parcels[parcels['use_code'].astype(str).str.startswith(('3', '4'))]

# Ensure both datasets have same CRS
commercial_parcels = commercial_parcels.to_crs(census.crs)

# Spatial join: assign each parcel to the tract it falls within
parcels_with_census = gpd.sjoin(
    commercial_parcels,
    census,
    how='left',
    predicate='within'
)

# Drop parcels that didn't match any census tract
parcels_with_census = parcels_with_census.dropna(subset=['pct_adv_deg'])

# Sort by advanced degree percentage (higher is better)
parcels_by_education = parcels_with_census.sort_values('pct_adv_deg', ascending=False)

# Get top 20 parcels in areas with highest educational attainment
top_parcels = parcels_by_education.head(20)

# Display results
result_ids = top_parcels['ml'].tolist()
print(f"Top 20 commercial parcels in areas with highest percentage of advanced degrees:")
print(f"Parcel IDs: {result_ids}")

Top 20 commercial parcels in areas with highest percentage of advanced degrees:
Parcel IDs: ['160-14', '160-59', '133-20', '160-84', '160-85', '133-51', '160-48', '160-11', '133-54', '133-52', '160-77', '160-76', '133-15', '160-67', '160-69', '160-58', '133-14', '160-83', '133-56', '125-73']


In [28]:
import geopandas as gpd
from shapely.geometry import Point

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')

# Define subway station locations (approximated for Cambridge)
subway_stations = [
    Point(-71.1189, 42.3736),  # Harvard Square
    Point(-71.1031, 42.3656),  # Central Square
    Point(-71.0865, 42.3625),  # Kendall/MIT
    Point(-71.1226, 42.3782),  # Porter Square
    Point(-71.1429, 42.3954)   # Alewife
]
subway_gdf = gpd.GeoDataFrame(geometry=subway_stations, crs=parcels.crs)

# Convert to projected CRS for accurate distance measurement
parcels_proj = parcels.to_crs(epsg=26986)
subway_gdf_proj = subway_gdf.to_crs(epsg=26986)

# Define commercial use codes consistently
commercial_use_codes = ['300', '301', '302', '303', '304', '305', '310',
                       '320', '325', '326', '330', '340', '345', '346']

# Filter for commercial parcels
commercial_parcels = parcels_proj[parcels_proj['use_code'].astype(str).str.startswith(('3', '4'))]

# Filter for vacant commercial parcels (using vacant land use codes)
vacant_codes = ['130', '131', '132', '390', '391', '392', '393']
vacant_commercial = commercial_parcels[commercial_parcels['use_code'].astype(str).isin(vacant_codes)]

# If no explicit vacant codes found, use undeveloped parcels
if len(vacant_commercial) == 0:
    # Look for commercial parcels with no buildings
    if 'building_count' in commercial_parcels.columns:
        vacant_commercial = commercial_parcels[commercial_parcels['building_count'] == 0]
    else:
        # If no building count, use all commercial parcels as fallback
        vacant_commercial = commercial_parcels
        print("Warning: No explicit vacant parcel data found. Using all commercial parcels.")

# Filter for parcels larger than 3000 sq ft
large_vacant_commercial = vacant_commercial[vacant_commercial['land_area'] > 3000]

# Create 800m buffer around subway stations
buffers = subway_gdf_proj.geometry.buffer(800)
union_buffer = buffers.union_all()

# Find large vacant commercial parcels within 800m of subway stations
result_parcels = large_vacant_commercial[large_vacant_commercial.geometry.intersects(union_buffer)]

# Convert back to original CRS
result_parcels = result_parcels.to_crs(parcels.crs)

# Sort by parcel ID for consistency
result_parcels = result_parcels.sort_values('ml')

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} vacant commercial parcels > 3000 sq ft within 800m of subway stations")
print(f"Parcel IDs: {result_ids}")

Found 13 vacant commercial parcels > 3000 sq ft within 800m of subway stations
Parcel IDs: ['11-46', '119-21', '173-35', '189-103', '267.1-282', '267E-17', '267E-290', '267E-291', '267E-292', '267F-393', '41-28', '70-10', '95-1']


In [30]:
import geopandas as gpd

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')
poi = gpd.read_file('cambridge_poi_processed.geojson')

# Define retail use codes consistently
retail_use_codes = ['320', '321', '322', '323', '324', '325', '326', '330', '331']

# Filter for retail parcels
retail_parcels = parcels[parcels['use_code'].astype(str).isin(retail_use_codes)]

# Convert to projected CRS for accurate distance measurement
retail_parcels_proj = retail_parcels.to_crs(epsg=26986)

# Filter retail businesses from POI data
retail_business_types = ['restaurant', 'clothing_store', 'department_store', 'grocery_store',
                         'convenience_store', 'furniture_store', 'hardware_store', 'electronics_store']
retail_businesses = poi[poi['business_type'].isin(retail_business_types)]
retail_businesses_proj = retail_businesses.to_crs(epsg=26986)

# Count nearby retail businesses for each parcel
def count_nearby_retail(parcel_geom, retail_geoms, buffer_distance=300):
    buffered_geom = parcel_geom.buffer(buffer_distance)
    count = sum(1 for geom in retail_geoms if buffered_geom.contains(geom))
    return count

# Calculate retail count for each retail parcel
retail_geoms = retail_businesses_proj['geometry'].tolist()
retail_parcels_proj['nearby_retail'] = retail_parcels_proj['geometry'].apply(
    lambda geom: count_nearby_retail(geom, retail_geoms)
)

# Filter for parcels with fewer than 2 competing retail businesses
result_parcels = retail_parcels_proj[retail_parcels_proj['nearby_retail'] < 2]

# Convert back to original CRS
result_parcels = result_parcels.to_crs(parcels.crs)

# Sort by parcel ID for consistency
result_parcels = result_parcels.sort_values('ml')

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} retail parcels with <2 competing retail businesses within 300m")
print(f"Parcel IDs: {result_ids}")

Found 6 retail parcels with <2 competing retail businesses within 300m
Parcel IDs: ['189-4', '195-59', '228-55', '234-178', '267D-259', '268B-15']


In [34]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

# Load data
parcels = gpd.read_file('cambridge_parcels.geojson')
poi = gpd.read_file('cambridge_poi_processed.geojson')
spend_data = pd.read_csv('cambridge_spend_processed.csv')

# Define approximate Massachusetts Avenue location
# Using points from POI data that are known to be on Mass Ave
mass_ave_points = [
    (-71.107894, 42.367672),  # Near Tanjore restaurant on Mass Ave
    (-71.10564, 42.367132),   # Near Fire Department on Mass Ave
    (-71.10144, 42.363676)    # Near The Boston Lamb Takedown on Mass Ave
]
mass_ave = LineString(mass_ave_points)

# Create GeoDataFrame for Mass Ave
mass_ave_gdf = gpd.GeoDataFrame(geometry=[mass_ave], crs=parcels.crs)

# Convert to projected CRS for accurate distance measurement
parcels_proj = parcels.to_crs(epsg=26986)
mass_ave_proj = mass_ave_gdf.to_crs(epsg=26986)

# Define commercial use codes consistently
commercial_use_codes = ['300', '301', '302', '303', '304', '305', '310',
                        '320', '325', '326', '330', '340', '345', '346']

# Filter for commercial parcels
commercial_parcels = parcels_proj[parcels_proj['use_code'].astype(str).str.startswith(('3', '4'))]

# Create 1km buffer around Mass Ave
mass_ave_buffer = mass_ave_proj.buffer(1000)

# Find commercial parcels within 1km of Mass Ave
parcels_near_mass_ave = commercial_parcels[commercial_parcels.geometry.intersects(mass_ave_buffer.iloc[0])]

# Combine POI with spending data
poi_with_spend = poi.merge(
    spend_data,
    left_on='PLACEKEY',
    right_on='PLACEKEY',
    how='left'
)

# Fill NaN values in spending with 0
if 'RAW_TOTAL_SPEND' in poi_with_spend.columns:
    poi_with_spend['RAW_TOTAL_SPEND'] = poi_with_spend['RAW_TOTAL_SPEND'].fillna(0)
else:
    poi_with_spend['RAW_TOTAL_SPEND'] = 0
    print("Warning: No spend data found. Setting all spend to 0.")

# Convert POI to projected CRS
poi_with_spend_proj = poi_with_spend.to_crs(epsg=26986)

# Calculate area spending for each parcel
def calculate_area_spending(parcel_geom, poi_geoms, spend_values, radius=200):  # 200m radius
    buffered_geom = parcel_geom.buffer(radius)
    total_spend = 0

    for i, poi_geom in enumerate(poi_geoms):
        if poi_geom is not None and buffered_geom.contains(poi_geom):
            total_spend += spend_values[i]

    return total_spend

# Calculate spending around each parcel
poi_geoms = poi_with_spend_proj['geometry'].tolist()
spend_values = poi_with_spend_proj['RAW_TOTAL_SPEND'].tolist()

# Calculate area spending for each parcel near Mass Ave
area_spending = []
for idx, row in parcels_near_mass_ave.iterrows():
    spend = calculate_area_spending(row['geometry'], poi_geoms, spend_values)
    area_spending.append(spend)

parcels_near_mass_ave['area_spending'] = area_spending

# Sort by area spending (higher is better)
result_parcels = parcels_near_mass_ave.sort_values('area_spending', ascending=False)

# Convert back to original CRS
result_parcels = result_parcels.to_crs(parcels.crs)

# Display results
result_ids = result_parcels['ml'].tolist()
print(f"Found {len(result_parcels)} commercial parcels within 1km of Mass Ave")
print(f"Parcel IDs (sorted by consumer spending): {result_ids[:30]}")

Found 328 commercial parcels within 1km of Mass Ave
Parcel IDs (sorted by consumer spending): ['128-71', '90-184', '106-124', '90-125', '90-133', '90-185', '90-155', '91-118', '91-200', '93-74', '93-76', '93-75', '93-73', '93-72', '93-78', '93-80', '93-79', '93-99', '90-193', '91-208', '106-123', '90-70', '90-169', '106-109', '90-52', '90-55', '106-117', '92-88', '90-170', '90-161']


/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
